In this example tokens output by a transition function are distributed to the destination place that matches the token's type.

In [ ]:
from typing import Optional, Union
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization
from pydantic import BaseModel

from petritype.core.executable_graph_components import (
    ListPlaceNode, FunctionTransitionNode, ArgumentEdgeToTransition, ReturnedEdgeFromTransition,
    ExecutableGraphOperations
)
from petritype.core.rustworkx_graph import RustworkxGraph
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization
from IPython.display import display, clear_output

In [ ]:
def give_a_type(input_token: str) -> Union[str, int, float]:
    try:
        return int(input_token)
    except ValueError:
        try:
            return float(input_token)
        except ValueError:
            return input_token


In [ ]:
input_tokens = ["42", "3.14", "hello", "100", "world", "2.718"]

In [ ]:
executable_graph_nodes_and_edges = [
    ListPlaceNode("Inputs", str, input_tokens),

    ArgumentEdgeToTransition("Inputs", "Give a Type", "input_token"),
    FunctionTransitionNode("Give a Type", give_a_type),
    ReturnedEdgeFromTransition("Give a Type", "Strings"),
    ReturnedEdgeFromTransition("Give a Type", "Integers"),
    ReturnedEdgeFromTransition("Give a Type", "Floats"),

    ListPlaceNode("Strings", str, []),
    ListPlaceNode("Integers", int, []),
    ListPlaceNode("Floats", float, []),
]

In [ ]:
executable_graph = ExecutableGraphOperations.construct_graph(executable_graph_nodes_and_edges)
executable_pydigraph = RustworkxGraph.from_executable_graph(executable_graph)
display(SimpleGraphvizVisualization.graph(executable_pydigraph))

In [ ]:
async for step, diagram, transitions_fired in SimpleGraphvizVisualization.animate_execution_generator(
    executable_graph=executable_graph,
    executable_pydigraph=executable_pydigraph,
):
    clear_output(wait=True)
    print(f"Step {step}")
    display(diagram)
    print(f"Transitions fired: {transitions_fired}")
    if not transitions_fired:
        print("No more transitions to fire. Execution complete.")
        break
    time.sleep(1.0)
    plt.close()